BioPackで抽出した心電図情報のtxtをcsvにするプログラム

In [4]:
import numpy as np
import pandas as pd
from typing import List, Tuple
import os
import matplotlib.pyplot as plt
from scipy import signal

In [5]:
# 入力とする動画と動画のファイル名を取得
root_dir = "experimentData\\"
data_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
trueValueTxt_paths = []
print("動画ディレクトリ:", data_dirs)

for i in range(len(data_dirs)):
    data_dir = data_dirs[i]
    print(f"Processing video directory: {data_dir}")

    # 動画ファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.txt')]
    trueValueTxt_paths.extend(movie_files)

print(f"真値ファイルのパス: {trueValueTxt_paths}")

動画ディレクトリ: ['experimentData\\202509181755']
Processing video directory: experimentData\202509181755
真値ファイルのパス: ['experimentData\\202509181755\\202509181755.txt']


In [6]:
# 真値txtファイルを読み込み、データフレームに変換
for trueValueTxt_path in trueValueTxt_paths:
    print(f"Reading true value file: {trueValueTxt_path}")
    df = pd.read_csv(trueValueTxt_path, header=None, skiprows=6, names=['time', 'ppg'], encoding='shift_jis', sep='\\s+')
    save_path = trueValueTxt_path.replace('.txt', '.csv')
    print(f"Saving to CSV: {save_path}")
    df.to_csv(save_path, index=False, encoding='utf-8-sig')

Reading true value file: experimentData\202509181755\202509181755.txt
Saving to CSV: experimentData\202509181755\202509181755.csv


吉澤先生の精度比較ソフトウェアに入れるRRI_Simpleファイルを作るプログラム

In [7]:
def detect_peaks_from_ecg(ecg_signal, ecg_rate, method="neurokit"):
    """
    ECG信号からピークを検出し、そのインデックスの配列を返す関数

    Args:
        ecg_signal: ECG信号の配列
        ecg_rate: ECGのサンプリングレート
        method: ピーク検出方法 ("neurokit" または "scipy")

    Returns:
        peaks_indices: 検出されたピークのインデックスの配列
    """
    if method == "neurokit":
        # neurokit2を用いてピーク検出
        _ , info_dict = nk.ecg_process(ecg_signal, sampling_rate=ecg_rate, method="neurokit")
        peaks_indices = info_dict['ECG_R_Peaks']
    elif method == "scipy":
        # scipyを用いてピーク検出
        peaks_indices, _ = signal.find_peaks(ecg_signal, distance=ecg_rate * 0.6)
    else:
        raise ValueError("Unsupported method. Use 'neurokit' or 'scipy'.")

    return peaks_indices

In [8]:
# 入力とする動画と動画のファイル名を取得
root_dir = "experimentData\\"
data_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
movie_paths = []
movie_names = []
true_value_csv_array = []
print("動画ディレクトリ:", data_dirs)

for i in range(len(data_dirs)):
    data_dir = data_dirs[i]

    # 動画ファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.avi')]
    movie_paths.extend(movie_files)

    movie_name = os.path.basename(data_dir)
    movie_names.append(movie_name)

    # ppgファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
    true_value_csv_array = [f.replace('.avi', '.csv') for f in movie_paths if f.endswith('.avi')]
print(f"data_dirs: {data_dirs}")
print(f"movie_paths: {movie_paths}")
print(f"movie_names: {movie_names}")
print(f"true_value_csv_array: {true_value_csv_array}")

動画ディレクトリ: ['experimentData\\202509181755']
data_dirs: ['experimentData\\202509181755']
movie_paths: ['experimentData\\202509181755\\202509181755.avi']
movie_names: ['202509181755']
true_value_csv_array: ['experimentData\\202509181755\\202509181755.csv']


In [9]:
for i in range(len(movie_paths)):
    trueValueCSVPath = true_value_csv_array[i]
    trueValue = pd.read_csv(trueValueCSVPath)

    ecgRate = 1000
    trueValuePPG = trueValue['ppg'].values
    trueValuePPGTime = trueValue['time'].values

    trueValuePeaks = detect_peaks_from_ecg(trueValuePPG, ecgRate, method="scipy")
    trueValueRRI = np.diff(trueValuePPGTime[trueValuePeaks])  # RRIを計算
    trueValueRRITime = trueValuePPGTime[trueValuePeaks][1:]

    trueValueRRIDataFrame = pd.DataFrame(
        columns=['time', 'RRI', 'BPM']
    )
    trueValueRRIDataFrame['time'] = trueValueRRITime[1:-1]  # 最初と最後のピークは除外
    trueValueRRIDataFrame['RRI'] = trueValueRRI[1:-1]  # 最初と最後のRRIは除外
    trueValueRRIDataFrame['BPM'] = 60 / trueValueRRI[1:-1]  # RRIからBPMを計算

    rri_simple_path = os.path.join(data_dirs[i], 'RRI_Simple_' + movie_names[i] + '.csv')
    pd.DataFrame.to_csv(trueValueRRIDataFrame, rri_simple_path, index=False, encoding='utf-8-sig')